# Photogrammetry API

<div class="alert alert-block alert-warning">

<b>The Photogrammetry API is highly experimental and some features may not be available or function correctly at the moment. Please use this API with care, as we issue no guarantees this API won't break.
</b> 
</div>

In this tutorial we will go through how to use Cognite's Photogrammetry API to create contextualised 3D models using images.

The API reference is available here: [https://doc.cognitedata.com/api/playground/](https://doc.cognitedata.com/api/playground/)


### Background

In order to create contextualised 3D models we have developed a pipeline with several modular parts. First, images are uploaded as a zip file by the user. These images go through a quality control step, where low quality images (blurry) are filtered out. Next, the 3D reconstruction and tag detection steps are started. The former is done by using a method called photogrammetry, which uses overlapping images from a real-world object or scene to create a 3D model.

The tag detection is done in two steps. First we use Convolution Neural Networks (CNN) for detection of tags, and thereafter optical character recognition (OCR) is used for recognizing the tag text. In the last step, the image pixel coordinates extracted by the tag detector are mapped to 3D locations.


<img src="images/pipeline.png" width="600" align="center">

## Step-by-step example

The photogrammetry API is currently only enabled for a few selected tenants, including **mltest**. If you want access from other tenants, please contact us via the **#ml-vision** Slack channel or open a pull request in this repo.

First, we make the imports we'll need for this tutorial and use an API key to authenticate. Make sure that you have first set the API key as an environment variable.

In [ ]:
import os
from urllib.request import urlretrieve
from cognite.client import CogniteClient
from cognite.client.stable.assets import Asset

BASEURL = "/api/playground/vision"
os.environ["COGNITE_DISABLE_GZIP"] = "1"
client = CogniteClient(api_key=os.environ['COGNITE_API_KEY'])

### List Photogrammetry models

List all available Photogrammetry models:

In [ ]:
response = client.get(url=f"{BASEURL}/photogrammetry").json()
response

### Add assets to your Tenant

When contextualising the 3D model, we compare the detected tags suggested by the model, with actual asset tags obtained from either the assets in the current tenant or provided as an argument. If the current tenant does not have the asset you want to detect, you can add it using the following function:

In [ ]:
def add_asset(client, name):
    my_asset = Asset(name)
    assets_to_post = [my_asset]
    response = client.assets.post_assets(assets_to_post)
    
#add_asset(client, "50CX0001A")

### Create a Photogrammetry model

To create a Photogrammetry model you need to provide the `name` field. 

In [ ]:
name = "my_contextualised_model" # Your model name

If you want to contextualise the model (i.e. detect tags), you also have to set contextualise to `True`.

In [ ]:
contextualise_model = True

When contextualising a model, our service will detect tag ids which are compared against either a provided list of tag ids, or existing tags in the current tenant. 

In [ ]:
def create_photogrammetry_model(client, name, contextualise=False, tag_list=None):
    response = client.post(
        url= f"{BASEURL}/photogrammetry",
        body={"name": name, "contextualise": contextualise, "tagList": tag_list}
    )
    return response.json()


# Option 1) without contextualisation:
# photogrammetry_model = create_photogrammetry_model(client, name)

# Option 2) contextualisation from assets in the tenant:
# photogrammetry_model = create_photogrammetry_model(client, name, contextualise=contextualise_model) 

# Option 3) contextualisation from tag_list
photogrammetry_model = create_photogrammetry_model(client,
                                                   name, 
                                                   contextualise=contextualise_model, 
                                                   tag_list="['50CX0001A']")
photogrammetry_model

### Retrieve information about single a Photogrammetry model

Returns info about a specific photogrammetry model. Takes model id as argument.

In [ ]:
url = f"{BASEURL}/photogrammetry/{photogrammetry_model['id']}"
response = client.get(url=url).json()
response

### Upload images

To upload images, you need to provide the upload url for the Photogrammetry model and the filepath for the zip file containing the images. The reconstruction will start immediately after the upload. We have created a test dataset, which you can download from [here](https://drive.google.com/open?id=1VaIURFCuv0BT2ny_-wcz6Kucayj5qkIk) or you can use your own images (*remember to change the zip file path!*). 


<div class="alert alert-block alert-info"><b>
    
* The supported image formats are: jpeg, jpg, jpe, and png.
    
    
* The maximum number of images you can have in a model is 300.


* Maximum size of a single image: 128 MB 
</b></div>


In [ ]:
def upload_images(upload_url, zip_file_path):
    import requests
    headers = {"content-length": str(os.path.getsize(zip_file_path))}
    
    with open(zip_file_path, "rb") as file:
        requests.put(upload_url, data=file, headers=headers)
        
zip_file_path = "path/to/zipfile.zip" # CHANGE ME! 

upload_images(photogrammetry_model["uploadUrl"], zip_file_path)

### Retrieve the 3D file and Tag locations

To check whether your 3D model is ready, use `photogrammetry_model_info()` to see if the photogrammetry job is finished. If that is the case, `status` will be `SUCCESS` and you get a download URL link that can be used to download the 3D file (fbx file). The detected tags and their 3D location will be part of the response. 


<div class="alert alert-block alert-info"><b>
It takes around 15-20 minutes to process and create a 3D model using the dataset provided above.
</b></div>

In [ ]:
output_filename = "output.fbx"
url = f"{BASEURL}/photogrammetry/{photogrammetry_model['id']}"
response = client.get(url=url).json()


if response["status"] == "SUCCESS":
    download_url = response["download_url"]
    urlretrieve(download_url, output_filename)
    
response

In [ ]:
response["tags"]

### Delete the photogrammetry model

To avoid crowding it might be a good idea to remove old/unused photogrammetry models. Currently this can be done like this:

In [ ]:
url = f"{BASEURL}/photogrammetry/{photogrammetry_model['id']}"
response = client.delete(url=url).json()
response

## Useful links


* [Autodesk ReMake - How to Take Photos for Photogrammetry](https://www.youtube.com/watch?v=D7Torjkfec4)

* [The Art of Photogrammetry: How To Take Your Photos](https://www.tested.com/art/makers/460142-art-photogrammetry-how-take-your-photos/)



